# Handling dependencies

What if my "unit" uses system facilities?

- os.remove / datetime.now / etc.
- database access
- network connectivity
- user input

## Option 1: Dependency injection

Make the dependency part of the unit interface

In [1]:
import os, time
from collections import namedtuple
AuditRecord = namedtuple('AuditRecord', 'uid timestamp action')


def make_audit_record(action):
    return AuditRecord(os.getuid(), time.time(), action)

In [6]:
make_audit_record('update')

AuditRecord(uid=501, timestamp=1563911394.1138299, action='update')

Depends on: current user, current time (hard to test)

### With Dependency injection

In [7]:
def make_audit_record(action, time=time.time, getuid=os.getuid):
    return AuditRecord(getuid(), time(), action)    

In [8]:
def mytime():
    return 200

def mygetuid():
    return 1024

# make_audit_record('update', time=lambda: 200, getuid=lambda: 1024)
make_audit_record('update', time=mytime, getuid=mygetuid)

AuditRecord(uid=1024, timestamp=200, action='update')

Test can inject new versions of utcnow and getuid and test to ensure things work as expected. Unfortunately, we've "uglified" our interface.

## Option 2: Mocking and patching

The `unittest.mock` library provides the ability to patch dependencies:

In [9]:
# original implementation

def make_audit_record(action):
    return AuditRecord(os.getuid(), time.time(), action)

In [10]:
# Test code
import os, time
from unittest.mock import patch

with patch('os.getuid') as p_getuid, patch('time.time') as p_time:
    p_getuid.return_value = 1024
    p_time.return_value = 200
    rec = make_audit_record('update')
rec

AuditRecord(uid=1024, timestamp=200, action='update')

In [11]:
os.getuid(), time.time()

(501, 1563911618.258527)

# Database Mocking

Depending on how much DBMS-specific SQL you use, you *may* be able to use Python's builtin `sqlite` database as a mock:

In [12]:
!pip install pandas

You are using pip version 18.1, however version 19.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [13]:
import pandas as pd
import sqlite3

In [14]:
data = pd.read_csv('./data/closing-prices.csv', index_col=0, parse_dates=[0])
data.head()

,F,TSLA,GOOG,IBM,AAPL
2014-01-02,12.0890,150.10,NaN,157.6001,72.7741
2014-01-03,12.1438,149.56,NaN,158.5430,71.1756
2014-01-06,12.1986,147.00,NaN,157.9993,71.5637
2014-01-07,12.0420,149.36,NaN,161.1508,71.0516
2014-01-08,12.1673,151.28,NaN,159.6728,71.5019


## We can dump the dataframe to an in-memory (SQL) database

In [15]:
conn = sqlite3.connect(':memory:')
data.to_sql('prices', conn)

In [16]:
for row in conn.execute('SELECT * FROM prices LIMIT 5'):
    print(row)

('2014-01-02 00:00:00', 12.089, 150.1, None, 157.6001, 72.7741)
('2014-01-03 00:00:00', 12.1438, 149.56, None, 158.543, 71.1756)
('2014-01-06 00:00:00', 12.1986, 147.0, None, 157.9993, 71.5637)
('2014-01-07 00:00:00', 12.042, 149.36, None, 161.1508, 71.0516)
('2014-01-08 00:00:00', 12.1673, 151.28, None, 159.6728, 71.5019)


# We can read it back into a dataframe

In [17]:
data2 = pd.read_sql('SELECT * FROM prices WHERE IBM > 160', conn)
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 6 columns):
index    177 non-null object
F        177 non-null float64
TSLA     177 non-null float64
GOOG     167 non-null float64
IBM      177 non-null float64
AAPL     177 non-null float64
dtypes: float64(5), object(1)
memory usage: 8.4+ KB


In [18]:
data2.head()

,index,F,TSLA,GOOG,IBM,AAPL
0,2014-01-07 00:00:00,12.0420,149.36,NaN,161.1508,71.0516
1,2014-01-16 00:00:00,13.0990,170.97,NaN,160.3438,72.9215
2,2014-01-17 00:00:00,12.9346,170.01,NaN,161.4736,71.1348
3,2014-01-21 00:00:00,12.8484,176.68,NaN,160.0635,72.2400
4,2014-03-06 00:00:00,12.3674,252.94,NaN,160.2662,70.2476


# More mock examples

In [19]:
%%file data/test-examples/test8.py
import unittest
from unittest import mock


def echo_data(socket):
    data = socket.recv(42)
    socket.send(data)


class MyTest(unittest.TestCase):

    def test_send_recv(self):
        socket = mock.Mock()
        socket.recv.return_value = 'Some data'
        echo_data(socket)
        socket.send.assert_called_with('Some data')


Overwriting data/test-examples/test8.py


In [20]:
!python -m unittest data/test-examples/test8.py

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


In [21]:
from unittest import mock

In [22]:
m = mock.Mock()

In [23]:
m

<Mock id='4792729896'>

In [26]:
m.foo

<Mock name='mock.foo' id='4792733088'>

In [27]:
m()

<Mock name='mock()' id='4792862536'>

In [28]:
m.foo.return_value = 16
m.foo()

16

In [29]:
m.side_effect = ValueError

In [30]:
m()

ValueError: 

In [31]:
m[10]

TypeError: 'Mock' object is not subscriptable

In [32]:
m + m

TypeError: unsupported operand type(s) for +: 'Mock' and 'Mock'

In [33]:
m = mock.MagicMock()

In [34]:
m.__add__.return_value = 'foo'

In [35]:
m + m

'foo'

In [36]:
from datetime import datetime

In [37]:
datetime.utcnow()

datetime.datetime(2019, 7, 23, 20, 0, 18, 851845)

In [38]:
datetime.gmtnow()

AttributeError: type object 'datetime.datetime' has no attribute 'gmtnow'

In [39]:
mock_dt = mock.Mock()

In [40]:
mock_dt.utcnow()

<Mock name='mock.utcnow()' id='4793745816'>

In [41]:
mock_dt.gmtnow()

<Mock name='mock.gmtnow()' id='4793746488'>

In [42]:
mock_dt.gmtnow.side_effect = AttributeError

In [43]:
mock_dt.gmtnow()

AttributeError: 

In [44]:
mock_dt = mock.create_autospec(datetime)

In [45]:
mock_dt.utcnow()

<MagicMock name='mock.utcnow()' id='4793720728'>

In [46]:
mock_dt.gmtnow()

AttributeError: Mock object has no attribute 'gmtnow'

In [47]:
mock_dt.utcnow('this is a spurious argument')

<MagicMock name='mock.utcnow()' id='4793720728'>

In [48]:
mock_dt.utcnow.return_value = datetime(2011, 1, 1)

In [49]:
mock_dt.utcnow()

datetime.datetime(2011, 1, 1, 0, 0)

In [ ]:
# import boto3
# ec2 = boto3.resource('ec2')
ec2 = mock.Mock()
ec2.meta.client.describe_instances.return_value = {
    'Instances': [
        {'id': ...}
    ]
}

ec2.describe_instances()